In [8]:
%pip install wbdata
import wbdata
import pandas as pd
import math
from datetime import datetime

#Match World Bank age ranges to a given sex
def get_matching_indicators(age_range, sex):
    age_ranges = [f"{i:02d}{i+4:02d}" for i in range(0, 80, 5)] + ["80UP"]
    male_variables = {f"SP.POP.{age}.MA": f"Males {age}" for age in age_ranges}
    female_variables = {f"SP.POP.{age}.FE": f"Females {age}" for age in age_ranges}

    #Creates dictionary for sex variables
    variables = male_variables.copy()
    variables.update(female_variables)

    #Matches age to given sex, and includes "People"
    indicators = []
    for age in age_helper(age_range):
        if sex == "People":
            indicators.append(f"SP.POP.{age}.MA")
            indicators.append(f"SP.POP.{age}.FE")
        elif sex == "Males":
            indicators.append(f"SP.POP.{age}.MA")
        elif sex == "Females":
            indicators.append(f"SP.POP.{age}.FE")
    return indicators

#Converting given age range into World Bank ranges
def age_helper(age_range):
    lower_bound = 5 * (min(age_range) // 5) #Making the age ranges less discrete
    upper_bound = 5 * round(max(age_range) // 5)
    age_groups = ["{:02}{:02}".format(i, i+4) for i in range(lower_bound, min(80, upper_bound+1), 5)]

    if upper_bound >= 80:
        age_groups.append("80UP") #Gives data for people older than 80

    return age_groups

#Create population function
def population(year, sex, age_range, place):
    current_year = datetime.now().year
    if not (1960 <= year <= current_year):
        raise ValueError("Please choose another year that is between 1960 and this year.") #Error message for year
    if sex not in ["People", "Males", "Females"]:
        raise ValueError("Please choose from the following 3 options: 'People', 'Males', 'Females'") #Error message for sex

    indicators = get_matching_indicators(age_range, sex)

    try:
        data = wbdata.get_dataframe({ind: ind for ind in indicators}, country = place, date = f"{year}-01-01")
        if data is None or data.empty:
            print("No available data.") #Error message for null data
            return 0

        #Create proportions for the age ranges
        population_count = 0
        for ind in indicators:
            if not data[ind].isnull().iloc[0]:
                age_group = ind.split(".")[2]
                group_low = int(age_group[:2])

                if "UP" in age_group:
                    group_high = 200 #Defines oldest age as 200 (we don't have data on the oldest person)
                else:
                    group_high = int(age_group[2:])

                group_pop = data[ind].iloc[0]

                overlap_low = max(group_low, age_range[0])
                overlap_high = min(group_high, age_range[1])

                pop_prop = (overlap_high - overlap_low + 1) / (group_high - group_low + 1)

                population_count += group_pop * pop_prop

        return int(population_count)
        
    except Exception as e:
        print(f"Error fetching requested data: {e}") #Error message for any other error
        return 0

#This is what I will be testing in my unit tests
print(population(2022, "People", (1, 100), "USA"))
print(population(2000, "Males", (13, 27), "AUS"))
print(population(1960, "Females", (20, 55), "WLD"))
print(population(2022, "People", (1,100), "PSS"))

Note: you may need to restart the kernel to use updated packages.
319227461
2025713
658149127
2579498


In [7]:
#Unit Tests
if __name__ == "__main__":
    assert population(year=2022, sex='People', age_range=(1, 100), place='USA') > 300000000, "Too few people!"
    assert population(year=2000, sex='Males', age_range=(13, 27), place='AUS') > 2000000, "Too few males!"
    assert population(year=1960, sex='Females', age_range=(20, 55), place='WLD') > 650000000, "Too few females!"
    assert population(year=2022, sex='People', age_range=(1, 100), place='PSS') > 2500000, "Too few people!"
    print("All tests passed!")

All tests passed!
